This example makes a reactor geometry and a neutronics model. A homogenised material made of enriched lithium lead and eurofer is being used as the blanket material for this simulation in order to demonstrate the use of more complex materials.

In [ ]:
import neutronics_material_maker as nmm
import openmc
import openmc_dagmc_wrapper as odw

# makes a homogenised material for the blanket from lithium lead and
# eurofer
blanket_material = nmm.Material.from_mixture(
    fracs=[0.8, 0.2],
    materials=[
        nmm.Material.from_library(
            name='Pb842Li158',
            enrichment=90,
            temperature=500),
        nmm.Material.from_library(name='eurofer')
    ])

source = openmc.Source()
# sets the location of the source to x=0 y=0 z=0
source.space = openmc.stats.Point((450, 0, 0))
# sets the direction to isotropic
source.angle = openmc.stats.Isotropic()
# sets the energy distribution to 100% 14MeV neutrons
source.energy = openmc.stats.Discrete([14e6], [1])

# makes the neutronics material
neutronics_model = odw.NeutronicsModel(
    h5m_filename='dagmc.h5m,
    source=source,
    materials={
        'inboard_tf_coils_mat': 'copper',
        'center_column_shield_mat': 'WC',
        'divertor_mat': 'eurofer',
        'firstwall_mat': 'eurofer',
        'blanket_mat': blanket_material,  # use of homogenised material
        'blanket_rear_wall_mat': 'eurofer'},
    cell_tallies=['TBR'],
)

# starts the neutronics simulation
h5m_filename = neutronics_model.simulate(
    simulation_batches=2,
    simulation_particles_per_batch=10,  # this will need increasing to obtain accurate results
)

# extracts and processes the results from the statepoint file
odw.process_results(statepoint_filename=h5m_filename)

# prints the results to screen
print('TBR', neutronics_model.results['TBR'])